<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/AbdelMahm/CORONA/blob/master/pca_corona_evolution.ipynb"><img src="https://colab.research.google.com/img/colab_favicon_256px.png" />Run in Google Colab</a>
  </td>
</table>

In [4]:
import sys
import sklearn
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import urllib.request
from sklearn import preprocessing
from sklearn import pipeline
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets

In [5]:
import urllib.request
corona_dataset_path = os.path.join("corona_dataset", "")
##pull data from ourworldindata
download_path = "https://covid.ourworldindata.org/data/ecdc/"
os.makedirs(corona_dataset_path, exist_ok=True)
for filename in ("total_cases.csv", "total_deaths.csv"):
    print("Downloading", filename)
    url = download_path + filename
    urllib.request.urlretrieve(url, corona_dataset_path + filename)

In [6]:
df_confirmed = pd.read_csv(corona_dataset_path + "total_cases.csv", thousands=',')
df_deaths = pd.read_csv(corona_dataset_path + "total_deaths.csv", thousands=',')
#df_recovered = pd.read_csv(corona_dataset_path + "time_series_19-covid-Recovered.csv", thousands=',')

In [7]:
df_confirmed.head()

,date,World,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,...,United Kingdom,United States,United States Virgin Islands,Uruguay,Uzbekistan,Vatican,Venezuela,Vietnam,Zambia,Zimbabwe
0,2019-12-31,27,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,2020-01-01,27,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,2020-01-02,27,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,2020-01-03,44,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,2020-01-04,44,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [8]:
df_deaths.head()

,date,World,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,...,United Kingdom,United States,United States Virgin Islands,Uruguay,Uzbekistan,Vatican,Venezuela,Vietnam,Zambia,Zimbabwe
0,2019-12-31,0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,2020-01-01,0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,2020-01-02,0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,2020-01-03,0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,2020-01-04,0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [10]:
# transpose
df_confirmed = df_confirmed.T
df_deaths = df_deaths.T

#drop date row
df_confirmed = df_confirmed.drop(['date'])
df_deaths = df_deaths.drop(['date'])

#fill NANs
df_confirmed = df_confirmed.fillna(0)
df_deaths = df_deaths.fillna(0)



In [11]:
df_confirmed.head(500)

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
World,27.0,27.0,27.0,44.0,44.0,59.0,59.0,59.0,59.0,59.0,...,213258.0,242472.0,271241.0,305270.0,338298.0,378113.0,417061.0,467929.0,528019.0,591971.0
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,22.0,24.0,24.0,34.0,40.0,42.0,75.0,75.0,91.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,59.0,70.0,70.0,76.0,89.0,100.0,123.0,146.0,174.0,186.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,73.0,82.0,94.0,94.0,102.0,189.0,231.0,264.0,305.0,305.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,53.0,75.0,75.0,88.0,113.0,133.0,164.0,188.0,224.0,267.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,3.0,4.0
Anguilla,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
Antigua and Barbuda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,3.0,3.0,7.0,7.0
Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,97.0,128.0,158.0,225.0,266.0,301.0,387.0,502.0,589.0,690.0
Armenia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,115.0,122.0,136.0,160.0,190.0,194.0,265.0,290.0,329.0,372.0


In [115]:
test = df_confirmed.loc[:, 79].abs()
print(test)


#df_confirmed.index.get_loc('Morocco')


Afghanistan    22.0
Albania        59.0
Algeria        73.0
Andorra        53.0
Angola          0.0
               ... 
Vatican         1.0
Venezuela      33.0
Vietnam        76.0
Zambia          2.0
Zimbabwe        0.0
Name: 79, Length: 195, dtype: float64


In [66]:
## Drop China and World

#df_confirmed = df_confirmed.drop(['China'])
#df_confirmed = df_confirmed.drop(['World'])

In [116]:
def pca_move(country, compare_with, constant_size, day):

    #===============================
    #Confirmed
    #===============================
    X = df_confirmed.iloc[:, 0:0+day+1]
    X = StandardScaler().fit_transform(X)
    pca = PCA(n_components=2)
    pc_confirmed = pca.fit_transform(X)
    df_pc_confirmed = pd.DataFrame(data = pc_confirmed, columns = ['PC1', 'PC2']).abs()
    
    MAX = np.max(df_pc_confirmed)
    MIN = np.min(df_pc_confirmed)
    #print(pca.explained_variance_ratio_)

    #===============================
    #Deaths
    #===============================
    X = df_deaths.iloc[:, 0:0+day+1]
    X = StandardScaler().fit_transform(X)
    pca = PCA(n_components=2)
    pc_deaths = pca.fit_transform(X)
    df_pc_deaths= pd.DataFrame(data = pc_deaths, columns = ['PC1', 'PC2'])

    #==============================
    #plots configs
    #==============================
    plt.figure(1, figsize = (10,10))
    plt.ylim(-60 , 60)
    plt.xlim(-40 , 200)

    plt.xlabel('PC1', fontsize = 15)
    plt.ylabel('PC2', fontsize = 15)
    plt.title('2 component PCA', fontsize = 20)
    
    #==============================
    #setup options
    #==============================
    confirmed_size = df_confirmed.iloc[:, 0+day+1]
    deaths_size    = df_deaths.iloc[:, 0+day+1]
    
    df_pca_all = [df_pc_confirmed]#, df_pc_deaths]
    sizes_all  = [confirmed_size ]#, deaths_size]
    targets    = ['confirmed'    ]#, 'deaths']
    colors     = ['r'            ]#, 'k']
    pads       = [0              ]#, 10]
    
    #==============================
    #main loop
    #==============================
    idx_country = df_confirmed.index.get_loc(country)

    if compare_with == 'all':
        for target, color, df_pca, size, add in zip(targets, colors, df_pca_all, sizes_all, pads):
            pc1 = df_pca.loc[:, 'PC1']
            pc2 = df_pca.loc[:, 'PC2']
            plt.scatter(pc1 + add, 
                        pc2, 
                        c = color, 
                        s = 30 if constant_size else size/2 )
            
            plt.annotate(country[0:2], (df_pca.loc[idx_country, 'PC1']+.03 + add, df_pca.loc[idx_country, 'PC2']+.03))
            #plt.annotate(compare_with[0:2], (df_pca.loc[idx_compare_with, 'PC1']+.03 + add, df_pca.loc[idx_compare_with, 'PC2']+.03))

    else:
        idx_compare_with = df_confirmed.index.get_loc(compare_with)

        for target, color, df_pca, size, add in zip(targets, colors, df_pca_all, sizes_all, pads):
            pc1 = df_pca.loc[[idx_country, idx_compare_with], 'PC1']
            pc2 = df_pca.loc[[idx_country, idx_compare_with], 'PC2']
            plt.scatter(pc1 + add, 
                        pc2, 
                        c = color, 
                        s = 30 if constant_size else size )
            
            plt.annotate(country, (df_pca.loc[idx_country, 'PC1']+.03 + add, df_pca.loc[idx_country, 'PC2']+.03))
            plt.annotate(compare_with, (df_pca.loc[idx_compare_with, 'PC1']+.03 + add, df_pca.loc[idx_compare_with, 'PC2']+.03))
   
    #==============================
    #plots
    #==============================
    lgnd = plt.legend(targets)
    lgnd.legendHandles[0]._sizes = [10]
    #lgnd.legendHandles[1]._sizes = [10]
    title = 'day:' + str(day)
    plt.title(title)
    plt.grid()

    plt.show()

In [117]:
%matplotlib inline

interactive_plot = interactive(pca_move,                                
                               country =['Morocco', 'Spain', 'Italy', 'Spain', 'Iran', 'Japan', 'United States'],
                               compare_with =['Morocco', 'Spain', 'Italy', 'Spain', 'Iran', 'Japan', 'United States', 'all'],
                               constant_size = [True, False],
                               day =(1, 87)) #
interactive_plot